In [ ]:
import numpy as np
import mne
import os

# 설정값
sfreq = 200                      # 샘플링 주파수
duration_sec = 7200              # 총 길이 7200초 (2시간)
times = np.arange(0, duration_sec, 1/sfreq)

# bipolar 채널명
bipolar_chs = [
    'Fp1-F3', 'F3-C3', 'C3-P3', 'P3-O1',
    'Fp2-F4', 'F4-C4', 'C4-P4', 'P4-O2',
    'Fp1-F7', 'F7-T3', 'T3-T5', 'T5-O1',
    'Fp2-F8', 'F8-T4', 'T4-T6', 'T6-O2',
    'Fz-Cz', 'Cz-Pz'
]

n_channels = len(bipolar_chs)

rng = np.random.default_rng(seed=42)

# 빈 데이터 생성
data = np.zeros((n_channels, len(times)))

# 주파수 대역별 파형 생성 (SRSE의 특징 반영)
freq_powers = {
    'delta': (2, 20e-6),   # 강한 delta 파형 (2Hz)
    'theta': (6, 10e-6),   # 뚜렷한 theta 파형 (6Hz)
    'alpha': (10, 4e-6),   # 감소한 alpha 파형 (10Hz)
    'beta': (20, 2e-6),    # 더 감소한 beta 파형 (20Hz)
    'gamma': (40, 1e-6)    # 매우 감소한 gamma 파형 (40Hz)
}

# 각 채널에 주파수별 파형 추가
for freq_name, (freq_hz, amplitude) in freq_powers.items():
    wave = amplitude * np.sin(2 * np.pi * freq_hz * times)
    for ch in range(n_channels):
        data[ch] += wave

# 랜덤 노이즈 추가
data += 0.5e-6 * rng.normal(size=(n_channels, len(times)))

# Seizure 이벤트 설정 (SRSE의 특성 반영하여 여러번 길게 발생)
seizure_events = [
    (600, 780),    # 10분~13분 (3분 지속)
    (1800, 1920),  # 30분~32분 (2분 지속)
    (3000, 3420),  # 50분~57분 (7분 지속)
    (4800, 5100),  # 80분~85분 (5분 지속)
    (6600, 6900)   # 110분~115분 (5분 지속)
]

seizure_freq = 6    # theta 범위의 seizure (흔한 ictal frequency)
seizure_amp = 50e-6 # 진폭을 크게 설정하여 뚜렷한 발작 신호 재현

for start_sec, end_sec in seizure_events:
    start_idx = int(start_sec * sfreq)
    end_idx = int(end_sec * sfreq)
    seizure_time = times[:end_idx - start_idx]
    
    # seizure waveform (주파수 고정)
    seizure_wave = seizure_amp * np.sin(2 * np.pi * seizure_freq * seizure_time)
    
    # temporal 및 frontal 채널에서 seizure 표현
    seizure_channels = ['Fp1-F7', 'F7-T3', 'T3-T5', 'Fp2-F8', 'F8-T4', 'T4-T6']
    for ch_name in seizure_channels:
        ch_idx = bipolar_chs.index(ch_name)
        data[ch_idx, start_idx:end_idx] += seizure_wave

# Raw 객체 생성
info = mne.create_info(ch_names=bipolar_chs, sfreq=sfreq, ch_types='eeg')
raw_srse_example = mne.io.RawArray(data, info)

# ECG, EOG 예시 데이터 생성
ecg_data = 1e-6 * np.sin(2 * np.pi * 1.2 * times)  # ECG-like (~1.2Hz)
eog_data1 = 1e-6 * rng.normal(size=len(times))     # EOG 노이즈
eog_data2 = 1e-6 * rng.normal(size=len(times))

ecg_eog_data = np.vstack([ecg_data, eog_data1, eog_data2])

ecg_eog_info = mne.create_info(['ECG', 'EOG1', 'EOG2'], sfreq=sfreq, ch_types=['ecg', 'eog', 'eog'])
ecg_eog_raw = mne.io.RawArray(ecg_eog_data, ecg_eog_info)

# EEG 데이터와 합치기
raw_srse_example.add_channels([ecg_eog_raw], force_update_info=True)

# EDF 형식으로 저장
output_fname = 'example_bipolar_eeg.edf'
mne.export.export_raw(output_fname, raw_srse_example, fmt='edf', overwrite=True)
